In [1]:
from pynq import Overlay

In [2]:
overlay = Overlay('../overlay/dft3.bit')

In [3]:
dft=overlay.dft_0

In [4]:
from pynq import allocate
sample_len = 1024
reInput_buffer = allocate(shape=(sample_len,), dtype='float32')
imgInput_buffer = allocate(shape=(sample_len,), dtype='float32')
reOut_buffer = allocate(shape=(sample_len,), dtype='float32')
imgOut_buffer = allocate(shape=(sample_len,), dtype='float32')

In [5]:
import numpy as np
real_In =np.array(np.arange(0,1024,1.0))
image_In =np.zeros((1,1024))

np.copyto(reInput_buffer, real_In)
np.copyto(imgInput_buffer,image_In)

In [6]:
dft.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  real_sample_1 = Register(real_sample=write-only),
  real_sample_2 = Register(real_sample=write-only),
  imag_sample_1 = Register(imag_sample=write-only),
  imag_sample_2 = Register(imag_sample=write-only),
  real_op_1 = Register(real_op=write-only),
  real_op_2 = Register(real_op=write-only),
  imag_op_1 = Register(imag_op=write-only),
  imag_op_2 = Register(imag_op=write-only)
}

In [7]:
dft.register_map.real_sample_1 = reInput_buffer.physical_address
dft.register_map.imag_sample_1 = imgInput_buffer.physical_address
dft.register_map.real_op_1 = reOut_buffer.physical_address
dft.register_map.imag_op_1 = imgOut_buffer.physical_address

In [8]:
# dft.s_axi_control.write(0x10,reInput_buffer.physical_address)
# dft.s_axi_control.write(0x1c,imgInput_buffer.physical_address)
# dft.s_axi_control.write(0x28,reOut_buffer.physical_address)
# dft.s_axi_control.write(0x34, imgOut_buffer.physical_address)


In [9]:
imgOut_buffer

PynqBuffer([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [10]:
reOut_buffer

PynqBuffer([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [11]:
import time

dft.register_map.CTRL.AP_START=1
start_time = time.time()
while True:
    reg = dft.register_map.CTRL.AP_DONE
    if reg != 1:
        break
end_time = time.time()

print("耗时：{}s".format(end_time - start_time))

耗时：0.0009293556213378906s


In [12]:
reOut_buffer

PynqBuffer([ 5.2377600e+05, -5.1200800e+02, -5.1199518e+02, ...,
            -5.1197339e+02, -5.1199518e+02, -5.1200800e+02], dtype=float32)

In [13]:
imgOut_buffer

PynqBuffer([      0.   ,  166885.69 ,   83441.93 , ...,  -55627.137,
             -83441.93 , -166885.69 ], dtype=float32)

In [14]:
print("%0.15f",reOut_buffer[0])

%0.15f 523776.0


In [15]:
print(real_In)

[0.000e+00 1.000e+00 2.000e+00 ... 1.021e+03 1.022e+03 1.023e+03]
